## get table mappings 

In [ ]:
import gspread
import pandas as pd
from google.oauth2.service_account import Credentials

# Scopes for Sheets + Drive
scopes = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

# Load credentials
creds = Credentials.from_service_account_file(
    "../uw_backend/secrets/service_account_key.json",
    scopes=scopes
)

# Authorize gspread
client = gspread.authorize(creds)

# Open the spreadsheet and worksheet
spreadsheet = client.open_by_key("1sQfkcakio2ppSPG6tYFRMbqiPMqCUWub0QlJrLLV3pw")
worksheet = spreadsheet.worksheet("Model Variable Mapping")

# ✅ Get all values including formulas (not evaluated values)
raw_data = worksheet.get_all_values(value_render_option='FORMULA')

# ✅ Convert to DataFrame (first row is headers)
df = pd.DataFrame(raw_data[1:], columns=raw_data[0])

# Preview
df

,section,field_key,location,start_month_location,end_month_location
0,General Property Assumptions,Asking Price,=Assumptions!O3,,
1,General Property Assumptions,Acquisition Price,=Assumptions!G20,,
2,General Property Assumptions,Model Start Date,=Assumptions!F2,,
3,General Property Assumptions,Closing on Property,=Assumptions!G21,,
4,General Property Assumptions,Gross Square Feet,=Assumptions!G22,,
5,General Property Assumptions,Share of Equity from Sponsor,=Assumptions!K3,,
6,Leasing Assumptions,Rehab time,=Assumptions!G51,,
7,Leasing Assumptions,Lease-up Time,=Assumptions!G52,,
8,Leasing Assumptions,Bad Debt,=Assumptions!G54,,
9,Leasing Assumptions,Vacancy,=Assumptions!G55,,


In [ ]:
import time
import re
from gspread.utils import a1_to_rowcol

def clean_number(value):
    if isinstance(value, str):
        value = value.strip().replace("'", "").replace(",", "")
        if value.endswith('%'):
            return float(value.strip('%')) / 100
        try:
            return float(value)
        except ValueError:
            return value
    return value

def update_inputs(ws, row_val, col_val):
    url = f"https://sheets.googleapis.com/v4/spreadsheets/{ws.spreadsheet.id}/values:batchUpdate"
    body = {
        "valueInputOption": "RAW",
        "data": [
            {"range": f"{ws.title}!E62", "values": [[clean_number(row_val)]]},
            {"range": f"{ws.title}!G20", "values": [[clean_number(col_val)]]},
        ],
    }
    response = ws.spreadsheet.client.session.request("POST", url=url, json=body)
    response.raise_for_status()

def get_range_string(corner, num_rows, num_cols):
    col_letter = corner[0]
    row_number = int(corner[1:])
    end_col = chr(ord(col_letter) + num_cols - 1)
    end_row = row_number + num_rows - 1
    return f"{corner}:{end_col}{end_row}"

def run_dual_output_table(
    worksheet,
    purchase_price_cell="E62",
    exit_cap_cell="G20",
    IRR_cell="E69",
    MOIC_cell="E70",
    IRR_left_corner="E76",
    MOIC_left_corner="E86",
    num_rows=5,
    num_cols=5
):
    # Auto-calculate row/col header ranges
    start_row, start_col = a1_to_rowcol(IRR_left_corner)
    row_header_col = chr(ord('A') + start_col - 2)  # One col to the left
    col_header_row = start_row - 1                 # One row above

    row_header_range = f"{row_header_col}{start_row}:{row_header_col}{start_row + num_rows - 1}"
    col_header_start_col_letter = chr(ord('A') + start_col - 1)
    col_header_end_col_letter = chr(ord(col_header_start_col_letter) + num_cols - 1)
    col_header_range = f"{col_header_start_col_letter}{col_header_row}:{col_header_end_col_letter}{col_header_row}"

    # Save original input values
    original_E62 = worksheet.acell(purchase_price_cell).value
    original_G20 = worksheet.acell(exit_cap_cell).value

    # Read input headers
    row_inputs_raw = worksheet.get(row_header_range)
    row_inputs = [r[0] for r in row_inputs_raw if r and len(r) > 0 and r[0] != '']
    print(row_header_range)
    print(row_inputs)
    col_inputs = worksheet.get(col_header_range)[0]

    irr_grid = []
    moic_grid = []

    for row_val in row_inputs:
        irr_row = []
        moic_row = []
        for col_val in col_inputs:
            update_inputs(worksheet, row_val, col_val)
            time.sleep(0.2)
            results = worksheet.get(f"{IRR_cell}:{MOIC_cell}")
            irr_result = results[0][0]
            moic_result = results[1][0]

            irr_row.append(irr_result)
            moic_row.append(moic_result)

        irr_grid.append(irr_row)
        moic_grid.append(moic_row)

    # Write output grids
    irr_range = get_range_string(IRR_left_corner, num_rows, num_cols)
    moic_range = get_range_string(MOIC_left_corner, num_rows, num_cols)
    worksheet.update(irr_range, irr_grid)
    worksheet.update(moic_range, moic_grid)

    # Restore inputs
    update_inputs(worksheet, original_E62, original_G20)

assumptions_ws = spreadsheet.worksheet("Assumptions")

run_dual_output_table(
    assumptions_ws,
    purchase_price_cell="E62",
    exit_cap_cell="G20",
    IRR_cell="E69",
    MOIC_cell="E70",
    IRR_left_corner="E76",
    MOIC_left_corner="E86",
    num_rows=5,
    num_cols=5
)




D76:D80
['5.50%', '5.75%', '6.00%', '6.25%', '6.50%']


/var/folders/07/sq03ndgd641brtd1_chp99z80000gn/T/ipykernel_40278/3988236566.py:89: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(irr_range, irr_grid)
/var/folders/07/sq03ndgd641brtd1_chp99z80000gn/T/ipykernel_40278/3988236566.py:90: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(moic_range, moic_grid)


D76:D80
['5.50%', '5.75%', '6.00%', '6.25%', '6.50%']


/var/folders/07/sq03ndgd641brtd1_chp99z80000gn/T/ipykernel_40278/3988236566.py:89: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(irr_range, irr_grid)
/var/folders/07/sq03ndgd641brtd1_chp99z80000gn/T/ipykernel_40278/3988236566.py:90: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(moic_range, moic_grid)


D76:D80
['5.50%', '5.75%', '6.00%', '6.25%', '6.50%']


/var/folders/07/sq03ndgd641brtd1_chp99z80000gn/T/ipykernel_40278/3988236566.py:89: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(irr_range, irr_grid)
/var/folders/07/sq03ndgd641brtd1_chp99z80000gn/T/ipykernel_40278/3988236566.py:90: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(moic_range, moic_grid)


D76:D80
['5.50%', '5.75%', '6.00%', '6.25%', '6.50%']


/var/folders/07/sq03ndgd641brtd1_chp99z80000gn/T/ipykernel_40278/3988236566.py:89: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(irr_range, irr_grid)
/var/folders/07/sq03ndgd641brtd1_chp99z80000gn/T/ipykernel_40278/3988236566.py:90: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(moic_range, moic_grid)


D76:D80
['5.50%', '5.75%', '6.00%', '6.25%', '6.50%']


/var/folders/07/sq03ndgd641brtd1_chp99z80000gn/T/ipykernel_40278/3988236566.py:89: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(irr_range, irr_grid)
/var/folders/07/sq03ndgd641brtd1_chp99z80000gn/T/ipykernel_40278/3988236566.py:90: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(moic_range, moic_grid)


D76:D80
['5.50%', '5.75%', '6.00%', '6.25%', '6.50%']


/var/folders/07/sq03ndgd641brtd1_chp99z80000gn/T/ipykernel_40278/3988236566.py:89: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(irr_range, irr_grid)
/var/folders/07/sq03ndgd641brtd1_chp99z80000gn/T/ipykernel_40278/3988236566.py:90: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(moic_range, moic_grid)


D76:D80
['5.50%', '5.75%', '6.00%', '6.25%', '6.50%']


/var/folders/07/sq03ndgd641brtd1_chp99z80000gn/T/ipykernel_40278/3988236566.py:89: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(irr_range, irr_grid)
/var/folders/07/sq03ndgd641brtd1_chp99z80000gn/T/ipykernel_40278/3988236566.py:90: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(moic_range, moic_grid)


D76:D80
['5.50%', '5.75%', '6.00%', '6.25%', '6.50%']


/var/folders/07/sq03ndgd641brtd1_chp99z80000gn/T/ipykernel_40278/3988236566.py:89: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(irr_range, irr_grid)
/var/folders/07/sq03ndgd641brtd1_chp99z80000gn/T/ipykernel_40278/3988236566.py:90: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(moic_range, moic_grid)


D76:D80
['5.50%', '5.75%', '6.00%', '6.25%', '6.50%']


/var/folders/07/sq03ndgd641brtd1_chp99z80000gn/T/ipykernel_40278/3988236566.py:89: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(irr_range, irr_grid)
/var/folders/07/sq03ndgd641brtd1_chp99z80000gn/T/ipykernel_40278/3988236566.py:90: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(moic_range, moic_grid)
